In [10]:
valid_codes = '''AUS,AUT,BEL,BGR,HRV,CHL,CZE,DNK,GBR,EST,FIN,FRA,DEU,GRC,HUN,ISL,ISR,ITA,LVA,LTU,LUX,NLD,NZL,NOR,POL,PRT,KOR,RUS,SVN,SVK,ESP,CHE,SWE,TWN,USA'''.split(",")

In [11]:
#these are the countries we have ex_mort data for from country codes text file
#thank you Alex
print(valid_codes)

['AUS', 'AUT', 'BEL', 'BGR', 'HRV', 'CHL', 'CZE', 'DNK', 'GBR', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'ISR', 'ITA', 'LVA', 'LTU', 'LUX', 'NLD', 'NZL', 'NOR', 'POL', 'PRT', 'KOR', 'RUS', 'SVN', 'SVK', 'ESP', 'CHE', 'SWE', 'TWN', 'USA']


In [12]:
import pandas as pd
import numpy as np
ex_mort = pd.read_csv("clean_combined.csv")
safety = pd.read_csv("covid-stringency-index.csv")
#only use countries that we have ex_mort data for
safety = safety[safety["Code"].isin(valid_codes)]
#only use weeks from the pandemic
ex_mort = ex_mort[ex_mort["years_relative"] >=0]


#convert to datetime to grab week number and year
safety["Day"] =  pd.to_datetime(safety["Day"])

In [13]:
safety["week"] = safety["Day"].dt.week
safety['year'] = safety["Day"].dt.year

/tmp/ipykernel_1733/2192045428.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  safety["week"] = safety["Day"].dt.week


In [14]:
si_df = safety.groupby(["Code", "week", "year"])["stringency_index"].mean().reset_index()
si_df = si_df.rename(columns={"Code": "country"})
si_df.head(10)

,country,week,year,stringency_index
0,AUS,1,2021,70.830000
1,AUS,1,2022,43.451429
2,AUS,2,2021,55.090000
3,AUS,2,2022,44.244286
4,AUS,3,2021,55.090000
5,AUS,3,2022,64.221429
6,AUS,4,2020,1.853333
7,AUS,4,2021,57.338571
8,AUS,4,2022,64.680000
9,AUS,5,2020,7.940000


In [15]:
#put the stringency index averages per week into the rest of the data
df = pd.merge(ex_mort, si_df)
df.head(10)

,date,country,week,year,DTotal,mov_avg,years_relative,pred_deaths,excess_mortality,em_per_capita,...,new_cases,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population,mov_avg_cases,new_cases_per_capita,normalized_cases,stringency_index
0,2020-02-02,CHL,4,2020,1964.0,2006.714286,0,1875.178571,131.535714,6.846410e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2020-02-02,SWE,4,2020,1730.0,1791.000000,0,1861.609774,-70.609774,-6.949672e-06,...,1.0,NaN,0.0,0.098,NaN,10160159.0,1.0,9.842366e-08,3.057224e-06,0.0
2,2020-02-02,ESP,4,2020,9954.0,9006.428571,0,9470.538910,-464.110338,-8.516927e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2020-02-02,DNK,4,2020,1033.0,1079.714286,0,1063.016484,16.697802,1.990192e-07,...,10.0,NaN,0.0,0.012,NaN,83900471.0,10.0,1.191888e-07,3.702230e-06,0.0
4,2020-02-02,LVA,4,2020,555.0,572.000000,0,555.143985,16.856015,1.814230e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,2020-02-02,GRC,4,2020,2774.0,2668.285714,0,2680.928571,-12.642857,-1.853598e-07,...,2.0,NaN,0.0,0.029,NaN,68207114.0,2.0,2.932245e-08,9.108106e-07,0.0
6,2020-02-02,BEL,4,2020,2204.0,2263.142857,0,2273.806015,-10.663158,-9.166826e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,2020-02-02,LUX,4,2020,86.0,84.571429,0,84.822932,-0.251504,-4.902112e-09,...,12.0,NaN,0.0,0.078,NaN,51305184.0,7.0,1.364385e-07,4.238035e-06,0.0
8,2020-02-02,KOR,4,2020,6490.0,6244.000000,0,6382.580952,-138.580952,-2.703333e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,2020-02-02,HUN,4,2020,2723.0,2580.000000,0,2673.120301,-93.120301,-8.979131e-06,...,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0


In [17]:
#loading vaccination data, converting date to week/year
owidVac = pd.read_csv("vaccinations.csv")
owidVac = owidVac[owidVac["iso_code"].isin(valid_codes)]

owidVac["date"] = pd.to_datetime(owidVac["date"])
owidVac["week"] = owidVac["date"].dt.isocalendar().week
owidVac['year'] = owidVac["date"].dt.year

In [18]:
#sort vaccination data, prepare for merge
vc = owidVac.groupby(["iso_code", "week", "year"])["total_vaccinations"].max().reset_index()
vc = vc.rename(columns={"iso_code": "country"})
vc.head(10)

,country,week,year,total_vaccinations
0,AUS,1,2022,44086608.0
1,AUS,2,2022,46034466.0
2,AUS,3,2022,48010195.0
3,AUS,4,2022,49607239.0
4,AUS,5,2022,51098236.0
5,AUS,6,2022,52308370.0
6,AUS,7,2021,20.0
7,AUS,7,2022,53224792.0
8,AUS,8,2021,34630.0
9,AUS,8,2022,53963467.0


In [19]:
#merge vaccination data into dataset
df = pd.merge(df, vc)
df.head(10)

,date,country,week,year,DTotal,mov_avg,years_relative,pred_deaths,excess_mortality,em_per_capita,...,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population,mov_avg_cases,new_cases_per_capita,normalized_cases,stringency_index,total_vaccinations
0,2020-12-13,NOR,49,2020,788.0,786.285714,0,770.623308,15.662406,8.389373e-06,...,272.0,87.0,11526.921,610.0,1866934.0,2.999571e+03,0.001607,0.049907,56.020000,0.0
1,2020-12-13,LVA,49,2020,669.0,593.000000,0,506.451880,86.548120,9.315264e-06,...,2924.0,82.0,37326.122,749.0,9291000.0,6.696143e+03,0.000721,0.022387,53.700000,1.0
2,2020-12-20,DNK,50,2020,1095.0,1068.714286,0,1010.758242,57.956044,6.907714e-07,...,22634.0,4294.0,16177.037,NaN,83900471.0,1.386594e+05,0.001653,0.051335,49.998571,2.0
3,2020-12-20,NOR,50,2020,840.0,792.571429,0,777.776692,14.794737,7.924617e-06,...,352.0,90.0,13864.979,775.0,1866934.0,3.543429e+03,0.001898,0.058955,56.020000,11.0
4,2020-12-20,LVA,50,2020,711.0,612.428571,0,510.478947,101.949624,1.097294e-05,...,3004.0,100.0,38694.005,942.0,9291000.0,8.536143e+03,0.000919,0.028538,53.700000,3.0
5,2020-12-20,USA,50,2020,81952.0,71991.142857,0,57378.114286,14613.028571,4.389416e-05,...,305114.0,19284.0,49928.031,109689.0,332915074.0,1.246826e+06,0.003745,0.116332,71.760000,31762.0
6,2020-12-27,ISR,51,2020,1007.0,925.571429,0,888.566165,37.005263,3.841046e-06,...,8282.0,948.0,31671.670,7097.0,9634162.0,2.949200e+04,0.003061,0.095086,70.505714,7462.0
7,2020-12-27,USA,51,2020,82864.0,74772.857143,0,57984.942857,16787.914286,5.042702e-05,...,324546.0,16252.0,54561.498,114648.0,332915074.0,1.320415e+06,0.003966,0.123198,71.760000,1220381.0
8,2020-12-27,DNK,51,2020,1273.0,1110.714286,0,1021.131868,89.582418,1.067722e-06,...,27110.0,3897.0,18286.167,NaN,83900471.0,1.394307e+05,0.001662,0.051620,51.850000,8.0
9,2020-12-27,RUS,51,2020,51148.0,51733.857143,0,30928.603759,20805.253383,5.504472e-04,...,25474.0,1721.0,31942.562,18376.0,37797000.0,1.016249e+05,0.002689,0.083516,47.690000,28500.0


In [ ]:
df.plot.scatter("stringency_index", "em_per_capita")

In [ ]:
import seaborn
import matplotlib.pyplot as plt
seaborn.set(style='ticks')
seaborn.relplot(data=df, x='stringency_index', y='em_per_capita', hue='year',  aspect=1.61)
plt.show()

In [ ]:
year2 = df[df["year"] == 2021]

In [ ]:
seaborn.relplot(data=year2, x='stringency_index', y='em_per_capita', aspect=1.61)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(df[["year", "em_per_capita", "normalized_cases", "stringency_index"]])
plt.show()

In [ ]:
seaborn.relplot(data=year2, x='stringency_index', y='normalized_cases', aspect=1.61)
plt.show()

In [ ]:
from sklearn import linear_model
df = df[df['normalized_cases'].notna()]
reg = linear_model.LinearRegression()
testx = df[["stringency_index"]]
testy = df["normalized_cases"]
reg.fit(testx, testy)

In [ ]:
reg.score(testx, testy)

In [ ]:
reg.intercept_

In [ ]:
y_pred = reg.predict(testx)
plt.scatter(testx, testy, color ='b')
plt.plot(testx, y_pred, color ='k')
  
plt.show()

In [ ]:
ridgereg = linear_model.Ridge(0.5)
ridgereg.fit(testx,testy)

In [ ]:
import datetime
ex_mort = pd.read_csv("clean_combined.csv")
safety = pd.read_csv("covid-stringency-index.csv")
#only use countries that we have ex_mort data for
safety = safety[safety["Code"].isin(valid_codes)]
#only use weeks from the pandemic
ex_mort = ex_mort[ex_mort["years_relative"] >=0]

for i in range(6):
    d = datetime.timedelta(days=i*7)
    #convert to datetime to grab week number and year
    safety["Day"] =  pd.to_datetime(safety["Day"])+d

    safety["week"] = safety["Day"].dt.week
    safety['year'] = safety["Day"].dt.year
    si_df = safety.groupby(["Code", "week", "year"])["stringency_index"].mean().reset_index()
    si_df = si_df.rename(columns={"Code": "country"})
    si_df.head(10)

    df = pd.merge(ex_mort, si_df)
    seaborn.relplot(data=df, x='stringency_index', y='em_per_capita', hue='year', aspect=1.61).set(title = "{} day offset".format(i*7))
    plt.show()